In [1]:
%load_ext autoreload
%autoreload 2

# Load Packages

In [2]:
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
%matplotlib inline

from numpy_fracdiff import fracdiff
import numpy as np

from statsmodels.tsa.stattools import adfuller
import scipy.optimize

# Load Data

In [3]:
with np.load('data/demo1.npz') as data:
    X = data['px']
    #t = data['t']

In [4]:
X = X[1:]  # chop 01-Jan
len(X)

782

# Example
Transform all $X$ time series with `fracdiff` by the fractal order $d=0.3$.
Truncate at 100 (i.e. chop the first 100 NANs too).

In [5]:
Z = fracdiff(X, order=0.3, truncation=100)
#np.isnan(Z[100:]).sum(axis=0)
Z = Z[100:]

Run the ADF test on all 0.3 fractal differentiated times series

In [6]:
for j in range(4):
    adf, pval, _, _, _, _ = adfuller(Z[:, j], regression='c', autolag='BIC')
    print("p-values: {:5.4f} | ADF: {:>6.3f}".format(pval, adf))

p-values: 0.4009 | ADF: -1.759
p-values: 0.0863 | ADF: -2.633
p-values: 0.0341 | ADF: -3.008
p-values: 0.0000 | ADF: -5.322


# Backtracking
For $d=1$ we usually get a stationary time series transform.
Thus, let start at $d=1$ and reduce towards $d=0$, 
and stop when the p-value exceeds the threshold $\alpha=0.01$.

In [7]:
%%time
x = X[:, 0]  # pick the 1st time series

n_steps = 30
order = 1
n_trunc = 100
alpha = 0.01
bestorder = order

for order in np.flip(np.arange(n_steps) / n_steps):
    z = fracdiff(x, order=order, truncation=n_trunc)
    _, pval, _, _, _, _ = adfuller(z[n_trunc:], regression='c', autolag='BIC')
    print("d: {:5.4f} | p-val: {:1.2E}".format(order, pval))
    if pval > alpha:
        break
    else:
        bestorder = order

print(f"best d={bestorder}")

d: 0.9667 | p-val: 0.00E+00
d: 0.9333 | p-val: 0.00E+00
d: 0.9000 | p-val: 0.00E+00
d: 0.8667 | p-val: 6.37E-29
d: 0.8333 | p-val: 1.05E-21
d: 0.8000 | p-val: 5.57E-20
d: 0.7667 | p-val: 4.12E-18
d: 0.7333 | p-val: 3.86E-16
d: 0.7000 | p-val: 6.56E-11
d: 0.6667 | p-val: 2.72E-09
d: 0.6333 | p-val: 8.89E-08
d: 0.6000 | p-val: 2.10E-06
d: 0.5667 | p-val: 2.05E-03
d: 0.5333 | p-val: 8.89E-03
d: 0.5000 | p-val: 2.91E-02
best d=0.5333333333333333
CPU times: user 759 ms, sys: 91.1 ms, total: 851 ms
Wall time: 470 ms


# Bisection
We will use difference between the ADF test p-value and required threshold $\alpha$.
The bisections requires the sign of this differences.

In [8]:
def loss_fn(d: float, alpha: float, x: np.array, n_trunc: int) -> float:
    z = fracdiff(x, order=d, truncation=n_trunc)
    _, pval, _, _, _, _ = adfuller(z[n_trunc:], regression='c', autolag='BIC')
    return pval - alpha  # for Bisection

In [9]:
loss_fn(0, alpha, x, n_trunc), loss_fn(1, alpha, x, n_trunc)

(0.966761902563547, -0.01)

Also note, that the `xtol` parameter doesn't need to be super precise.
We will abort if the p-value is 1% away from $\alpha$, i.e. `xtol=alpha*.01`

In [10]:
%%time
x = X[:, 0]  # pick the 1st time series
n_trunc = 100
alpha = 0.01

d = scipy.optimize.bisect(loss_fn, 0, 1, args=(alpha, x, n_trunc), xtol=alpha*.01)
d

CPU times: user 729 ms, sys: 76.4 ms, total: 806 ms
Wall time: 416 ms


0.53033447265625

The Ridder method is faster than the bisection method.

In [11]:
%%time
x = X[:, 0]  # pick the 1st time series
n_trunc = 100
alpha = 0.01

d = scipy.optimize.ridder(loss_fn, 0, 1, args=(alpha, x, n_trunc), xtol=alpha*.01)
d

CPU times: user 477 ms, sys: 52.2 ms, total: 529 ms
Wall time: 279 ms


0.5302962674827497

In [12]:
%%time
x = X[:, 0]  # pick the 1st time series
n_trunc = 100
alpha = 0.01

d = scipy.optimize.brenth(loss_fn, 0, 1, args=(alpha, x, n_trunc), xtol=alpha*.01)
d

CPU times: user 572 ms, sys: 61 ms, total: 633 ms
Wall time: 344 ms


0.5303438239411459

In [13]:
%%time
x = X[:, 0]  # pick the 1st time series
n_trunc = 100
alpha = 0.01

d = scipy.optimize.brentq(loss_fn, 0, 1, args=(alpha, x, n_trunc), xtol=alpha*.01)
d

CPU times: user 573 ms, sys: 61.7 ms, total: 634 ms
Wall time: 337 ms


0.5303403710498779

# Squared Errors
We will use the squared difference betweent the ADF test p-value and required threshold $\alpha$ as target function for a minimization problem.

$$
\min_d \; ({\rm pvalue}(d) - \alpha)^2
$$

In [14]:
def loss_fn(d: float, alpha: float, x: np.array, n_trunc: int) -> float:
    z = fracdiff(x, order=d, truncation=n_trunc)
    _, pval, _, _, _, _ = adfuller(z[n_trunc:], regression='c', autolag='BIC')
    return (pval - alpha)**2

The newton method is kind of unstable depending on the start value `x0` (e.g. 0.0 and 1.0 will fail)

In [15]:
%%time
x = X[:, 0]  # pick the 1st time series
n_trunc = 100
alpha = 0.01

d = scipy.optimize.newton(loss_fn, .1, args=(alpha, x, n_trunc), tol=(alpha*.01)**2)
d

CPU times: user 1.84 s, sys: 190 ms, total: 2.03 s
Wall time: 1.12 s


0.5303404343390861

In [16]:
"""
%%time
x = X[:, 0]  # pick the 1st time series
n_trunc = 100
alpha = 0.01

d = scipy.optimize.fmin(loss_fn, 0.1, args=(alpha, x, n_trunc), xtol=(alpha*.01)**2)
d
"""
print("doesn't with numba.jit decorator around frac_weights")

doesn't with numba.jit decorator around frac_weights


In [17]:
"""
%%time
x = X[:, 0]  # pick the 1st time series
n_trunc = 100
alpha = 0.01

d = scipy.optimize.fsolve(loss_fn, 0.1, args=(alpha, x, n_trunc), xtol=(alpha*.01)**2)
d
"""
print("doesn't with numba.jit decorator around frac_weights")

doesn't with numba.jit decorator around frac_weights


# Conclusion
The Ridder method seems to be fast way to find a fractal differentiation order.